In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras

from keras.datasets import mnist

tf.config.optimizer.set_jit(True)

2024-02-02 00:59:37.117945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 00:59:37.117971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 00:59:37.118825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
def normalize(img):
    img = tf.cast(img, tf.float32)
    img = tf.expand_dims(img, axis=-1)
    img = img / 255.0
    return img

keras_aug = tf.keras.Sequential([keras.layers.RandomRotation(0.10),
                                keras.layers.RandomZoom(0.2, 0.2),
                                keras.layers.RandomTranslation(0.15, 0.15)
                                ])
def augment(img):
    img = keras_aug(img)
    return img

def training_set(img, label):
    img = normalize(img)
    #img = augment(img)
    return img, label

def test_set(img, label):
    img = normalize(img)
    return img, label


In [7]:
SHUFFLE_SIZE = 512
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

training = tf.data.Dataset.from_tensor_slices((x_train, y_train))
training = training.map(training_set, num_parallel_calls=AUTOTUNE)
training = training.shuffle(SHUFFLE_SIZE)
training = training.batch(SHUFFLE_SIZE, num_parallel_calls=AUTOTUNE)
training = training.prefetch(AUTOTUNE)

test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test = test.map(test_set, num_parallel_calls=AUTOTUNE)
test = test.shuffle(SHUFFLE_SIZE)
test = test.batch(SHUFFLE_SIZE, num_parallel_calls=AUTOTUNE)
test = test.prefetch(AUTOTUNE)

## Design the model

In [8]:
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, Input
from keras.models import Model
from keras.layers import Layer

In [9]:
def ConvBlock(inputs, filters, momentum=0.99):
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization(momentum=momentum)(x)
    x = Activation('relu')(x)
    return x

In [10]:
def ResBlock(inputs, filters=[64, 64, 128], momentum=0.99):
    x = ConvBlock(inputs, filters[0], momentum=momentum)
    x = ConvBlock(x, filters[1], momentum=momentum)
    skip_connection = Conv2D(filters=filters[1], kernel_size=(1, 1), padding='same')(inputs)
    x = ConvBlock(skip_connection + x, filters=filters[2], momentum=momentum)
    x = MaxPooling2D()(x)
    return x

In [11]:
def ResNet(input_shape, num_classes, momentum=0.99):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), padding='same')(inputs)
    x = BatchNormalization(momentum=momentum)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = ResBlock(x, [64, 64, 128], momentum=momentum)
    x = ResBlock(x, [128, 128, 256], momentum=momentum)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(units=num_classes)(x)

    return Model(inputs=inputs, outputs=x)

In [12]:
model = ResNet((28, 28, 1), 10, momentum=0.6)
model.summary()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 28, 28, 64)           3200      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 28, 28, 64)           256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 28, 28, 64)           0         ['batch_normalization[0][0

In [13]:
history = model.fit(
    training,
    epochs=10,
    validation_data=test
)

Epoch 1/10


I0000 00:00:1706823000.537611   58341 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


118/118 [==============================] - 23s 76ms/step - loss: 0.9153 - accuracy: 0.7139 - val_loss: 0.1263 - val_accuracy: 0.9683
Epoch 2/10
118/118 [==============================] - 12s 99ms/step - loss: 0.1566 - accuracy: 0.9553 - val_loss: 0.0669 - val_accuracy: 0.9790
Epoch 3/10
118/118 [==============================] - 6s 51ms/step - loss: 0.0936 - accuracy: 0.9728 - val_loss: 0.0463 - val_accuracy: 0.9861
Epoch 4/10
118/118 [==============================] - 6s 51ms/step - loss: 0.0648 - accuracy: 0.9813 - val_loss: 0.0445 - val_accuracy: 0.9857
Epoch 5/10
118/118 [==============================] - 6s 51ms/step - loss: 0.0491 - accuracy: 0.9857 - val_loss: 0.0389 - val_accuracy: 0.9877
Epoch 6/10
118/118 [==============================] - 6s 51ms/step - loss: 0.0407 - accuracy: 0.9880 - val_loss: 0.0410 - val_accuracy: 0.9867
Epoch 7/10
118/118 [==============================] - 6s 51ms/step - loss: 0.0316 - accuracy: 0.9908 - val_loss: 0.0392 - val_accuracy: 0.9863
Epoch 8/

In [14]:
model.save('save.tf')

INFO:tensorflow:Assets written to: save.tf/assets


INFO:tensorflow:Assets written to: save.tf/assets
